In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data set

In [2]:
data = pd.DataFrame({'x1':[0.5,0.2], 'x2':[0.5,0.5], 'x3':[0.5,0.75],'x4':[0.5,0.9]}) 

# Model

In [3]:
# Defines a function which takes a data vector of values x_i and returns a vector of values 10*x_i
f = np.vectorize(lambda x: 10*x)

# Quantities to compar

## Unbinned Likelihood

In [4]:
def L_u(f,x):
    '''
    Calculates the unbinned Likelihood for a data vector x.
        f: vectorized function
        x: data vector
    
    Returns:
    -----
        Unbinned Likelihood = \prod f(x_i), i: from 1 to n
    '''
    return np.prod(f(x))

## Extended Likelihood

In [5]:
def L_e(f,x):
    '''
    Calculates the extended Likelihood for a data vector x.
        f: vectoreized function
        x: data vector
        l: normalization of the model
        n: number of events
    '''
    l = 5
    n = 4
    f1 = l**n*np.exp(-l)/np.math.factorial(n)
    f2 = np.prod(f(x)/l)
    return f1*f2

## Binned Poisson

In [6]:
def nu_i(k):
    '''
    Returns the result of calculating the integral of f in each one of the k regularly-spaced subintervals of [0,1].
    This is taken as the expectation value in each bin. 
        k: number of intervals
    '''
    F_list = np.linspace(0,1,k+1)**2*5 # F(x) = int f(x) = 5*x^2
    nu_list = [F_list[i + 1] - F_list[i] for i in range(len(F_list)-1)] 
    
    return np.array(nu_list)    

In [7]:
def n_i(x_data,k):
    '''
    Retruns the number of events in each bin for a given binning parameter k.
        x: data vector
        k: number of intervals
    '''
    intervals = np.linspace(0,1,k+1)
    
    return np.array([len([x for x in x_data if intervals[i] <= x <intervals[i+1]]) for i in range(len(intervals)-1)])    

In [8]:
def poisson_binned(x_data,k):
    return np.prod(np.exp(-nu_i(k))*nu_i(k)**n_i(x_data,k)/(np.vectorize(np.math.factorial)(n_i(x_data,k))))

# Comparison between sets and measures

## Set 0

In [9]:
x0 = data.loc[0]

In [10]:
L_u(f,x0)

625.0

In [11]:
L_e(f,x0)

0.1754673697678507

In [12]:
poisson_binned(x0,10)

2.569017760771112e-05

## Set 1

In [13]:
x1 = data.loc[1]

In [14]:
L_u(f,x1)

675.0

In [15]:
L_e(f,x1)

0.18950475934927882

In [16]:
poisson_binned(x1,10)

0.0006601082450666545

## Add results to table

In [17]:
data['L_unbin'] = [L_u(f,x0), L_u(f,x1)]

In [18]:
data['L_ext'] = [L_e(f,x0),L_e(f,x1)]

In [21]:
data['Poiss_binn (10 bins)'] = [poisson_binned(x0,10),poisson_binned(x1,10)]

In [22]:
data

,x1,x2,x3,x4,L_unbin,L_ext,Poiss_binn,Poiss_binn (10 bins)
0,0.5,0.5,0.50,0.5,625.0,0.175467,0.000026,0.000026
1,0.2,0.5,0.75,0.9,675.0,0.189505,0.000660,0.000660


In [27]:
print(data.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &   x1 &   x2 &    x3 &   x4 &  L\_unbin &     L\_ext &  Poiss\_binn (10 bins) \\
\midrule
0 &  0.5 &  0.5 &  0.50 &  0.5 &    625.0 &  0.175467 &              0.000026 \\
1 &  0.2 &  0.5 &  0.75 &  0.9 &    675.0 &  0.189505 &              0.000660 \\
\bottomrule
\end{tabular}

